# Download Data

In [1]:
import os
from scripts.download_dataset import download_from_uci

DATA_DIR = "/projects/aieng/diffusion_bootcamp/data/tabular"


NAME_URL_DICT_UCI = {
    "adult": "https://archive.ics.uci.edu/static/public/2/adult.zip",
    "default": "https://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip",
    "magic": "https://archive.ics.uci.edu/static/public/159/magic+gamma+telescope.zip",
    "shoppers": "https://archive.ics.uci.edu/static/public/468/online+shoppers+purchasing+intention+dataset.zip",
    "beijing": "https://archive.ics.uci.edu/static/public/381/beijing+pm2+5+data.zip",
    "news": "https://archive.ics.uci.edu/static/public/332/online+news+popularity.zip",
}

for name in NAME_URL_DICT_UCI.keys():
    download_from_uci(name, os.path.join(DATA_DIR, "raw_data"), NAME_URL_DICT_UCI)

Start processing dataset adult from UCI.
Aready downloaded.
Start processing dataset default from UCI.
Aready downloaded.
Start processing dataset magic from UCI.
Aready downloaded.
Start processing dataset shoppers from UCI.
Aready downloaded.
Start processing dataset beijing from UCI.
Aready downloaded.
Start processing dataset news from UCI.
Aready downloaded.


# Process Data

In [2]:
from scripts.process_dataset import process_data

INFO_PATH = "../code/data_info"

dataname = "adult"
# for name in ["adult", "default", "shoppers", "magic", "beijing", "news"]:
process_data(dataname, INFO_PATH, DATA_DIR)

adult (32561, 15) (16281, 15) (32561, 15)
Numerical (32561, 6)
Categorical (32561, 8)
Processing and Saving adult Successfully!
adult
Total 48842
Train 32561
Test 16281
Num 6
Cat 9


# Review Data

In [3]:
import pandas as pd


file_path = f"{DATA_DIR}/processed_data/{dataname}/train.csv"

df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head(20)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
5,37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K
6,49.0,Private,160187.0,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0.0,0.0,16.0,Jamaica,<=50K
7,52.0,Self-emp-not-inc,209642.0,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,45.0,United-States,>50K
8,31.0,Private,45781.0,Masters,14.0,Never-married,Prof-specialty,Not-in-family,White,Female,14084.0,0.0,50.0,United-States,>50K
9,42.0,Private,159449.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178.0,0.0,40.0,United-States,>50K


In [4]:
value = " ?"
if value in df.values:
    print(f"{value} exists in the DataFrame.")
else:
    print(f"{value} does not exist in the DataFrame.")

 ? exists in the DataFrame.


# Load Data

In [5]:
import os
import src
import json
import numpy as np

from src.data import make_dataset


# Replace 'path/to/your/file.json' with the actual path to your JSON file
real_data_path = f"{DATA_DIR}/processed_data/{dataname}"

In [6]:
# Open the JSON file and read its contents
with open(f"{real_data_path}/info.json", "r") as file:
    data_info = json.load(file)

# Display the JSON data
print(data_info)

{'name': 'adult', 'task_type': 'binclass', 'header': None, 'column_names': ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'capital.gain', 'capital.loss', 'hours.per.week', 'native.country', 'income'], 'num_col_idx': [0, 2, 4, 10, 11, 12], 'cat_col_idx': [1, 3, 5, 6, 7, 8, 9, 13], 'target_col_idx': [14], 'file_type': 'csv', 'data_path': '/projects/aieng/diffusion_bootcamp/data/tabular/raw_data/adult/adult.data', 'test_path': '/projects/aieng/diffusion_bootcamp/data/tabular/raw_data/adult/adult.test', 'column_info': {'0': {}, 'type': 'categorical', 'max': 99.0, 'min': 1.0, '2': {}, '4': {}, '10': {}, '11': {}, '12': {}, '1': {}, 'categorizes': [' <=50K', ' >50K'], '3': {}, '5': {}, '6': {}, '7': {}, '8': {}, '9': {}, '13': {}, '14': {}}, 'train_num': 32561, 'test_num': 16281, 'idx_mapping': {'0': 0, '1': 6, '2': 1, '3': 7, '4': 2, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, '10': 3, '11': 4, '12': 5, '13': 13, '1

In [7]:
config_path = f"../code/src/baselines/tabddpm/configs/{dataname}.toml"
raw_config = src.load_config(config_path)

print(raw_config)

{'parent_dir': 'configs/adult', 'model_save_path': 'ckpt/adult', 'sample_save_path': 'sampled/adult', 'real_data_path': 'Data/adult', 'num_numerical_features': 6, 'model_type': 'mlp', 'task_type': 'binclass', 'model_params': {'num_classes': 2, 'is_y_cond': False, 'rtdl_params': {'d_layers': [1024, 2048, 2048, 1024], 'dropout': 0.0}}, 'diffusion_params': {'num_timesteps': 1000, 'gaussian_loss_type': 'mse'}, 'train': {'main': {'steps': 100000, 'lr': 0.001809824563637657, 'weight_decay': 0.0005, 'batch_size': 4096}, 'T': {'seed': 0, 'normalization': 'quantile', 'num_nan_policy': 'mean', 'cat_nan_policy': None, 'cat_min_frequency': None, 'cat_encoding': None, 'y_policy': 'default'}}, 'sample': {'num_samples': 32561, 'batch_size': 10000, 'seed': 0}, 'eval': {'type': {'eval_model': 'mlp', 'eval_type': 'synthetic'}, 'T': {'seed': 0, 'normalization': 'quantile', 'num_nan_policy': None, 'cat_nan_policy': None, 'cat_min_frequency': None, 'cat_encoding': 'one-hot', 'y_policy': 'default'}}}


In [8]:
T = src.Transformations(**raw_config["train"]["T"])

dataset = make_dataset(
    real_data_path,
    T,
    task_type=data_info["task_type"],
    change_val=False,
)

data_path /projects/aieng/diffusion_bootcamp/data/tabular/processed_data/adult
No NaNs in numerical features, skipping


In [9]:
print("train dataset num shape: ", dataset.X_num["train"].shape)
print("test dataset num shape: ", dataset.X_num["test"].shape)

print("train dataset cat shape: ", dataset.X_cat["train"].shape)
print("test dataset cat shape: ", dataset.X_cat["test"].shape)

print(dataset.y_info)
print(dataset.task_type)
print(dataset.n_classes)

train dataset num shape:  (32561, 6)
test dataset num shape:  (16281, 6)
train dataset cat shape:  (32561, 9)
test dataset cat shape:  (16281, 9)
{'policy': 'default'}
binclass
None


In [10]:
K = np.array(dataset.get_category_sizes("train"))
print(K)
if len(K) == 0 or raw_config["train"]["T"]["cat_encoding"] == "one-hot":
    K = np.array([0])
print(K)

num_numerical_features = (
    dataset.X_num["train"].shape[1] if dataset.X_num is not None else 0
)
d_in = np.sum(K) + num_numerical_features
raw_config["model_params"]["d_in"] = d_in
print(d_in)

[ 2  9 16  7 15  6  5  2 42]
[ 2  9 16  7 15  6  5  2 42]
110


# Load Model

In [12]:
from src.baselines.tabddpm.pipeline import TabDDPM

device = "cuda"
tabddpm = TabDDPM(
    dataset=dataset,
    num_classes=K,
    **raw_config["diffusion_params"],
    real_data_path=real_data_path,
    model_type=raw_config["model_type"],
    model_params=raw_config["model_params"],
    num_numerical_features=num_numerical_features,
    device=device,
    )

{'num_classes': 2, 'is_y_cond': False, 'rtdl_params': {'d_layers': [1024, 2048, 2048, 1024], 'dropout': 0.0}, 'd_in': 110}
mlp
the number of parameters 11768942


# Train Model

In [13]:
model_save_path = f"/projects/aieng/diffusion_bootcamp/models/tabular/tabddpm/{dataname}"
tabddpm.train(
    **raw_config["train"]["main"],
    model_save_path=model_save_path,
)

Steps:  100000
Step 1/100000 MLoss: 2.1954 GLoss: 1.4004 Sum: 3.5957999999999997
Time:  1.3260135650634766
Step 2/100000 MLoss: 70.4426 GLoss: 91399.5469 Sum: 91469.9895
Time:  0.7472183704376221
Step 3/100000 MLoss: 1.7166 GLoss: 2.9138 Sum: 4.6304
Time:  1.5869474411010742
Step 4/100000 MLoss: 1.9694 GLoss: 1.3851 Sum: 3.3545
Time:  0.1681656837463379
Step 5/100000 MLoss: 1.8294 GLoss: 1.042 Sum: 2.8714
Time:  0.15831828117370605
Step 6/100000 MLoss: 1.6563 GLoss: 1.0761 Sum: 2.7324
Time:  0.13638949394226074
Step 7/100000 MLoss: 4.0731 GLoss: 21.1471 Sum: 25.2202
Time:  0.07087397575378418


KeyboardInterrupt: 

# Sample Data

In [16]:
sample_save_path=f"/projects/aieng/diffusion_bootcamp/synthetic_data/tabular/{dataname}/tabddpm.csv"
model_name= "model_100000.pt"
tabddpm.sample(
    num_samples=raw_config["sample"]["num_samples"],
    batch_size=raw_config["sample"]["batch_size"],
    disbalance=raw_config["sample"].get("disbalance", None),
    ckpt_path=f"{model_save_path}/{model_name}",
    sample_save_path=sample_save_path,
    ddim=True,
    steps=1000,
)

Loaded model from /projects/aieng/diffusion_bootcamp/models/tabular/tabddpm/adult/model_100000.pt
Sample using DDIM.
Sample timestep  999
Sample timestep  999
Sample timestep  999
Sample timestep  999
Shape torch.Size([32561, 15])
(32561, 9)
Sampling time: 249.61851859092712


# Synthetic Data

In [19]:
import pandas as pd


file_path = f"{DATA_DIR}/synthetic_data/{dataname}/tabddpm.csv"

df = pd.read_csv(file_path, na_values=[" ?"])

df.head(20)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,32.0,Private,171051.970,HS-grad,9.0,Divorced,Adm-clerical,Unmarried,Black,Female,0.000,0.0000,40.00000,United-States,<=50K
1,50.0,Private,257719.940,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Female,0.000,0.0000,45.00000,United-States,<=50K
2,30.0,Self-emp-not-inc,120122.140,Assoc-acdm,12.0,Divorced,Exec-managerial,Not-in-family,White,Male,0.000,0.0000,40.00000,United-States,<=50K
3,61.0,Private,269432.750,Some-college,10.0,Married-civ-spouse,Adm-clerical,Husband,White,Male,0.000,0.0000,35.00000,United-States,<=50K
4,66.0,Self-emp-inc,174884.500,HS-grad,9.0,Married-civ-spouse,Sales,Husband,White,Male,0.000,0.0000,40.00000,United-States,<=50K
5,32.0,Private,227510.360,HS-grad,9.0,Divorced,Craft-repair,Unmarried,Black,Female,0.000,0.0000,40.00000,United-States,<=50K
6,27.0,NaN,349068.220,HS-grad,9.0,Divorced,NaN,Not-in-family,Black,Male,0.000,0.0000,40.00000,United-States,<=50K
7,34.0,Private,233257.120,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,10183.372,0.0000,49.81715,United-States,>50K
8,19.0,Private,160105.080,HS-grad,9.0,Never-married,Other-service,Own-child,White,Female,0.000,0.0000,8.00000,United-States,<=50K
9,31.0,Private,209641.170,Bachelors,13.0,Married-civ-spouse,Sales,Husband,White,Male,0.000,1852.3655,55.00000,United-States,>50K


In [23]:
# value = " ?"
# if value in df.values:
#     print(f"{value} exists in the DataFrame.")
# else:
#     print(f"{value} does not exist in the DataFrame.")